# EVALUACIÓN DEL IMPACTO DEL SISTEMA DE RECOMENDACIONES MEJORADO EN LA CONVERSIÓN DE LA REGIÓN UE

## Descripción:

Evaluar estadísticamente los resultados de la prueba A/B "recommender_system_test" llevada a cabo para la tienda en línea internacional. La prueba compara al grupo de control (grupo A) con el grupo de sistema de recomendaciones mejorado (grupo B), enfocándose en nuevos usuarios de la región UE durante diciembre de 2020. 

El análisis incluye: 
1. El estudio del embudo de conversión (product_page -> product_cart -> purchase).
2. La aplicación de la prueba estadística Z para proporciones para determinar si el sistema de recomendaciones mejorado logró un aumento estadísticamente significativo de al menos el 10% en la conversión en cada etapa del embudo dentro de los 14 días posteriores a la inscripción. 

## 1. Inicialización

In [91]:
# Cargar todas las librerías necesarias.
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import seaborn as sns
import scipy.stats as stats
from statsmodels.stats.proportion import proportions_ztest

## 2. Carga de datos

In [92]:
# El calendario de eventos de marketing para 2020. 
df_calendar_events = pd.read_csv("/datasets/ab_project_marketing_events_us.csv")

# Los usuarios que se registraron entre el 07 y 21 de diciembre de 2020. 
df_new_users = pd.read_csv("/datasets/final_ab_new_users_upd_us.csv")

# Los eventos de los nuevos usuarios ocurridos entre el 07 de diciembre de 2020 y el 01 de enero de 2021.
df_new_users_events = pd.read_csv("/datasets/final_ab_events_upd_us.csv")

# Los grupos de los usuarios que participaron en la prueba. 
df_test_groups = pd.read_csv("/datasets/final_ab_participants_upd_us.csv")

## 3. Preprocesamiento de datos

In [93]:
# 1. Validar la estructura de cada DataFrame:
# a. Conocer el total de registros. 
# b. Conocer el total de columnas. 
# c. Comprobar los tipos de dato en cada columna. 
# d. Validar si existen valores ausentes. 
# 2. Mostrar los primeros 10 registros de cada DataFrame.

df_calendar_events.info()
print()
print(df_calendar_events)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes

                                name                   regions    start_dt  \
0           Christmas&New Year Promo             EU, N.America  2020-12-25   
1       St. Valentine's Day Giveaway  EU, CIS, APAC, N.America  2020-02-14   
2             St. Patric's Day Promo             EU, N.America  2020-03-17   
3                       Easter Promo  EU, CIS, APAC, N.America  2020-04-12   
4                  4th of July Promo                 N.America  2020-07-04   
5          Black Friday Ads Campaign  EU, CIS, APAC, N.America  2020-11-26   
6             Chinese New Year Promo                      APAC  2020-01-

Observaciones: 
1. El DataFrame df_calendar_events muestra la distribución temporal de los eventos de marketing que se tienen calendarizados para 2020.
2. Se tienen un total de 14 eventos de marketing.
3. Se consideran 4 regiones: EU, N.America, CIS, APAC. Dado que el análisis de la prueba A/B "recommender_system_test" se centra en la región UE, se considerarán únicamente aquellos eventos de marketing enfocados en esta región en particular. 
4. Las fechas de inicio y fin de campaña tienen el tipo de dato object. Se convertirán estas columnas de fecha al formato datetime.
5. El DataFrame df_calendar_events no tiene valores ausentes. 

In [94]:
# Convertir las columnas de inicio y fin de campaña a datetime. 
df_calendar_events["start_dt"] = pd.to_datetime(df_calendar_events["start_dt"])
df_calendar_events["finish_dt"] = pd.to_datetime(df_calendar_events["finish_dt"])

# Filtrar los eventos de marketing para obtener sólo los registros donde figura la región "EU"
df_calendar_events_filtered = df_calendar_events[df_calendar_events["regions"].str.contains("EU")]
print(df_calendar_events_filtered)

                               name                   regions   start_dt  \
0          Christmas&New Year Promo             EU, N.America 2020-12-25   
1      St. Valentine's Day Giveaway  EU, CIS, APAC, N.America 2020-02-14   
2            St. Patric's Day Promo             EU, N.America 2020-03-17   
3                      Easter Promo  EU, CIS, APAC, N.America 2020-04-12   
5         Black Friday Ads Campaign  EU, CIS, APAC, N.America 2020-11-26   
7  Labor day (May 1st) Ads Campaign             EU, CIS, APAC 2020-05-01   
8   International Women's Day Promo             EU, CIS, APAC 2020-03-08   

   finish_dt  
0 2021-01-03  
1 2020-02-16  
2 2020-03-19  
3 2020-04-19  
5 2020-12-01  
7 2020-05-03  
8 2020-03-10  


In [95]:
df_new_users.info()
print()
print(df_new_users.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58703 entries, 0 to 58702
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     58703 non-null  object
 1   first_date  58703 non-null  object
 2   region      58703 non-null  object
 3   device      58703 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB

            user_id  first_date     region   device
0  D72A72121175D8BE  2020-12-07         EU       PC
1  F1C668619DFE6E65  2020-12-07  N.America  Android
2  2E1BF1D4C37EA01F  2020-12-07         EU       PC
3  50734A22C0C63768  2020-12-07         EU   iPhone
4  E1BDDCE0DAFA2679  2020-12-07  N.America   iPhone
5  137119F5A9E69421  2020-12-07  N.America   iPhone
6  62F0C741CC42D0CC  2020-12-07       APAC   iPhone
7  8942E64218C9A1ED  2020-12-07         EU       PC
8  499AFACF904BBAE3  2020-12-07  N.America   iPhone
9  FFCEA1179C253104  2020-12-07         EU  Android


Observaciones: 
1. El dataframe df_new_users no tiene valores ausentes.
2. El dataframe df_new users contiene información sobre diferentes no solamente EU. Dado que el análisis de la prueba A/B "recommender_system_test" se centra en la región UE, se considerarán únicamente aquellos registros de usuarios en esta región en particular.
3. Las fechas de inscripción tienen el tipo de dato object. Se convertirá esta columna de fecha al formato datetime.

In [96]:
# Convertir la columna de fecha de inscripción al formato datetime. 
df_new_users["first_date"] = pd.to_datetime(df_new_users["first_date"])

# Filtrar los nuevos usuarios de la region de EU. 
df_new_users_filtered = df_new_users[df_new_users["region"] == "EU"]
print(df_new_users_filtered.head(10))

             user_id first_date region   device
0   D72A72121175D8BE 2020-12-07     EU       PC
2   2E1BF1D4C37EA01F 2020-12-07     EU       PC
3   50734A22C0C63768 2020-12-07     EU   iPhone
7   8942E64218C9A1ED 2020-12-07     EU       PC
9   FFCEA1179C253104 2020-12-07     EU  Android
11  084A22B980BA8169 2020-12-07     EU  Android
12  8ACC2420471B31E4 2020-12-07     EU       PC
13  5BE017E9C8CC42F8 2020-12-07     EU  Android
14  7B6452F081F49504 2020-12-07     EU   iPhone
15  0FC21E6F8FAA8DEC 2020-12-07     EU       PC


In [97]:
df_new_users_events.info()
print()
print(df_new_users_events.head(10))
print(df_new_users_events.sample(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     423761 non-null  object 
 1   event_dt    423761 non-null  object 
 2   event_name  423761 non-null  object 
 3   details     60314 non-null   float64
dtypes: float64(1), object(3)
memory usage: 12.9+ MB

            user_id             event_dt event_name  details
0  E1BDDCE0DAFA2679  2020-12-07 20:22:03   purchase    99.99
1  7B6452F081F49504  2020-12-07 09:22:53   purchase     9.99
2  9CD9F34546DF254C  2020-12-07 12:59:29   purchase     4.99
3  96F27A054B191457  2020-12-07 04:02:40   purchase     4.99
4  1FD7660FDF94CA1F  2020-12-07 10:15:09   purchase     4.99
5  831887FE7F2D6CBA  2020-12-07 06:50:29   purchase     4.99
6  6B2F726BFD5F8220  2020-12-07 11:27:42   purchase     4.99
7  BEB37715AACF53B0  2020-12-07 04:26:15   purchase     4.99
8  B5FA27F582227197  2020-12-07 01:

Observaciones: 
1. La única columna del dataframe df_new_users_events que tiene valores ausentes es la de detalles (details). Sin embargo, al analizar muestras aleatorias de este conjunto de datos, se observa que el único evento que presenta valores es el de compra (purchase). Esto tiene sentido, ya que de acuerdo con las especificaciones de los datos la columna details contiene el total en dólares (USD) de la compra. Por ende, no es necesario eliminar o llenar estos valores ausentes.
2. En la columna del nombre de tipo de evento figura el valor "login". No obstante, el análisis del embudo de conversión solicitado únicamente incluye los eventos product_page, product_cart y purchase, por lo que se discriminarán los registros para tipo de evento "login".
3. La columna de fecha y hora del evento (event_dt) tiene el tipo de dato object. Se convertirá esta columna al formato datetime.

In [98]:
# Convertir la columna de fecha y hora de evento al formato datetime. 
df_new_users_events["event_dt"] = pd.to_datetime(df_new_users_events["event_dt"])

# Filtrar los tipos de evento product_page, product_cart y purchase
df_new_users_events_filtered = df_new_users_events[df_new_users_events["event_name"] != "login"]
print(df_new_users_events_filtered.sample(10))
print()
print("Los tipos de evento son: ", df_new_users_events_filtered["event_name"].unique())

# Validar que la columna details no tenga valores ausents para los registros de compra (purchase)
purchase_registers = df_new_users_events_filtered[df_new_users_events_filtered["event_name"] == "purchase"]
print()
print("Valores ausentes en la columna details para registros de compra: " , purchase_registers["details"].isna().sum())

                 user_id            event_dt    event_name  details
171627  A9B681E82B51CF64 2020-12-17 22:17:34  product_page      NaN
226351  6620D9AFD8DF06BE 2020-12-24 08:38:27  product_page      NaN
84294   BDD4AC0CA848F33A 2020-12-16 00:20:43  product_cart      NaN
239702  239ED52B3E897B2E 2020-12-29 20:26:25  product_page      NaN
64791   B1EC454E754FB41E 2020-12-09 23:47:59  product_cart      NaN
169859  2A03965D143981E2 2020-12-17 05:08:24  product_page      NaN
60889   5D4120F42B0C8C23 2020-12-07 00:28:08  product_cart      NaN
114619  310102850E60716C 2020-12-26 22:32:56  product_cart      NaN
49773   903B7FB3C9AB7B72 2020-12-23 16:45:59      purchase     4.99
210528  2E915FD36B873A80 2020-12-22 11:12:55  product_page      NaN

Los tipos de evento son:  ['purchase' 'product_cart' 'product_page']

Valores ausentes en la columna details para registros de compra:  0


In [99]:
df_test_groups.info()
print()
print(df_test_groups.head(10))
print(df_test_groups.sample(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
dtypes: object(3)
memory usage: 340.6+ KB

            user_id group                  ab_test
0  D1ABA3E2887B6A73     A  recommender_system_test
1  A7A3664BD6242119     A  recommender_system_test
2  DABC14FDDFADD29E     A  recommender_system_test
3  04988C5DF189632E     A  recommender_system_test
4  4FF2998A348C484F     A  recommender_system_test
5  7473E0943673C09E     A  recommender_system_test
6  C46FE336D240A054     A  recommender_system_test
7  92CB588012C10D3D     A  recommender_system_test
8  B3A2485649E4A012     A  recommender_system_test
9  66FC298441D50783     A  recommender_system_test
                user_id group                  ab_test
7525   31AB656337375F00     A        interface_e

Observaciones:
1. El dataframe df_test_groups no tiene valores ausentes.
2. Existen registros para 2 pruebas A/B: "recommender_system_test" e "interface_eu_test".  Dado que el análisis a desarrollar es de la prueba A/B "recommender_system_test", se considerarán únicamente aquellos registros de esta prueba.

In [100]:
# Filtrar los registros de grupos para la prueba A/B "recommender_system_test". 
df_test_groups_filtered = df_test_groups[df_test_groups["ab_test"] == "recommender_system_test"]
print(df_test_groups_filtered.sample(10))
print()
print("Prueba A/B que considera el conjunto de datos: ", df_test_groups_filtered["ab_test"].unique())

               user_id group                  ab_test
3380  05CB67D01740EF6E     A  recommender_system_test
2758  3BA458C3A704EA03     A  recommender_system_test
3147  FCF9CA847EFA5A92     B  recommender_system_test
1399  F939B9720A300065     A  recommender_system_test
2140  28A47F8FF0E02694     A  recommender_system_test
2704  FB51E66D21EBF867     A  recommender_system_test
1222  9991812B20A2DBC8     A  recommender_system_test
2254  C845E42170625895     B  recommender_system_test
3051  DBFF94F143546A7B     B  recommender_system_test
530   8A2FACABB19EDC70     A  recommender_system_test

Prueba A/B que considera el conjunto de datos:  ['recommender_system_test']


In [101]:
# Unir el DataFrame de nuevos usuarios de la región UE con el DataFrame de eventos del embudo.
# Usar "inner" join para asegurarse de incluir los eventos cuyo user_id se encuentra en la lista filtrada de usuarios de EU. 

df_users_events_eu = df_new_users_filtered.merge(
    df_new_users_events_filtered,
    on = "user_id",
    how = "inner"
)

print(df_users_events_eu.head(10))


            user_id first_date region  device            event_dt  \
0  D72A72121175D8BE 2020-12-07     EU      PC 2020-12-07 21:52:10   
1  2E1BF1D4C37EA01F 2020-12-07     EU      PC 2020-12-07 09:05:47   
2  2E1BF1D4C37EA01F 2020-12-07     EU      PC 2020-12-10 04:13:53   
3  2E1BF1D4C37EA01F 2020-12-07     EU      PC 2020-12-12 17:54:57   
4  2E1BF1D4C37EA01F 2020-12-07     EU      PC 2020-12-07 09:05:46   
5  2E1BF1D4C37EA01F 2020-12-07     EU      PC 2020-12-10 04:13:53   
6  2E1BF1D4C37EA01F 2020-12-07     EU      PC 2020-12-12 17:54:57   
7  50734A22C0C63768 2020-12-07     EU  iPhone 2020-12-07 13:24:03   
8  50734A22C0C63768 2020-12-07     EU  iPhone 2020-12-08 03:27:16   
9  50734A22C0C63768 2020-12-07     EU  iPhone 2020-12-12 22:37:39   

     event_name  details  
0  product_page      NaN  
1  product_cart      NaN  
2  product_cart      NaN  
3  product_cart      NaN  
4  product_page      NaN  
5  product_page      NaN  
6  product_page      NaN  
7  product_cart      NaN

In [102]:
# Unir el DataFrame de nuevos usuarios en EU y sus eventos con el DataFrame de segmentación de grupos de la prueba "recommender_system_test"
df_users_events_groups_eu = df_users_events_eu.merge(
    df_test_groups_filtered,
    on = "user_id",
    how = "inner"
)

print(df_users_events_groups_eu.head(10))

            user_id first_date region   device            event_dt  \
0  D72A72121175D8BE 2020-12-07     EU       PC 2020-12-07 21:52:10   
1  DD4352CDCF8C3D57 2020-12-07     EU  Android 2020-12-07 15:32:54   
2  DD4352CDCF8C3D57 2020-12-07     EU  Android 2020-12-08 08:29:31   
3  DD4352CDCF8C3D57 2020-12-07     EU  Android 2020-12-10 18:18:27   
4  DD4352CDCF8C3D57 2020-12-07     EU  Android 2020-12-16 20:35:48   
5  DD4352CDCF8C3D57 2020-12-07     EU  Android 2020-12-18 23:57:25   
6  DD4352CDCF8C3D57 2020-12-07     EU  Android 2020-12-30 12:42:57   
7  831887FE7F2D6CBA 2020-12-07     EU  Android 2020-12-07 06:50:29   
8  831887FE7F2D6CBA 2020-12-07     EU  Android 2020-12-09 02:19:17   
9  831887FE7F2D6CBA 2020-12-07     EU  Android 2020-12-07 06:50:30   

     event_name  details group                  ab_test  
0  product_page      NaN     A  recommender_system_test  
1  product_page      NaN     B  recommender_system_test  
2  product_page      NaN     B  recommender_system_test

In [103]:
# Validar si existen registros duplicados idénticos. 
print("Número de registros duplicados idénticos: ", df_users_events_groups_eu.duplicated().sum())

Número de registros duplicados idénticos:  0


## 4. Análisis exploratorio de datos 

In [104]:
# Validar si los registros de nuevos usuarios abarcan del 07 al 21 de diciembre de 2020.
print("Los usuarios se registraron a partir de: ", df_users_events_groups_eu["first_date"].min())
print("Los usuarios se registraron hasta: ", df_users_events_groups_eu["first_date"].max())

Los usuarios se registraron a partir de:  2020-12-07 00:00:00
Los usuarios se registraron hasta:  2020-12-21 00:00:00


In [105]:
# Validar si un usuario registró el mismo tipo de evento en la misma fecha y hora.
print(df_users_events_groups_eu[["user_id", "event_dt", "event_name"]].duplicated().sum())

0


In [106]:
# Calcular el número de días transcurridos entre la fecha de inscripción y la ocurrencia de cada evento.}
df_users_events_groups_eu["days_since_registration"] = (df_users_events_groups_eu["event_dt"] -  df_users_events_groups_eu["first_date"]).dt.days

# Incluir únicamente los eventos que ocurrieron en un plazo de 14 días a partir de la fecha de inscripción. 
df_events_14_days = df_users_events_groups_eu[df_users_events_groups_eu["days_since_registration"] <= 14]
# print(df_events_14_days)

# Comparar el número de eventos totales y el número de eventos que ocurren en un plazo de 14 días a partir de la fecha de inscripción.
print("Eventos ocurridos sin límite de días: ", len(df_users_events_groups_eu))
print("Eventos ocurridos en 14 días: ", len(df_events_14_days))

Eventos ocurridos sin límite de días:  12413
Eventos ocurridos en 14 días:  12144


Observaciones: 
1. 269 eventos no ocurrieron dentro delos 14 días posteriores a la inscripción de nuevos usuarios.
2. Debido a que la prueba espera que los usuarios muestren una mejor conversión (10% al menos) dentro de los 14 días posteriores a la inscripción, estos 269 eventos se discriminarán del estudio de conversión de etapas del embudo product_page -> product_cart -> purchase, y de la evaluación de la prueba A/B "recommender_system_test".  


In [107]:
# Contar a los usuarios únicos en cada grupo de prueba. 
unique_users = df_events_14_days.groupby("group")["user_id"].nunique().reset_index()
unique_users.columns = ["user_id", "total_users"]
print("Usuarios únicos por grupo de prueba:")
print(unique_users)

Usuarios únicos por grupo de prueba:
  user_id  total_users
0       A         2120
1       B          666


In [108]:
# Validar si existen usuarios que hayan sido considerados tanto en el grupo A como en el B: 
# a. Agrupar el DataFrame por identificador de usuario (user_id).
# b. Contar el número de grupos de prueba únicos por usuario. 
users_grouped = df_events_14_days.groupby("user_id")["group"].nunique().reset_index()
users_grouped.columns = ["user_id", "groups_number"]
print(users_grouped)

# Filtrar a los usuarios que tienen más de un grupo de prueba: 
users_two_groups = users_grouped[users_grouped["groups_number"] > 1]
print(f"Número de usuarios que aparecen tanto en grupo A como B: {len(users_two_groups)}")
print(users_two_groups)

               user_id  groups_number
0     001064FEAAB631A1              1
1     0010A1C096941592              1
2     003DF44D7589BBD4              1
3     005E096DBD379BCF              1
4     006E3E4E232CE760              1
...                ...            ...
2781  FF5B24BCE4387F86              1
2782  FF825C1D791989B5              1
2783  FF8CF7057415EB29              1
2784  FFAE9489C76F352B              1
2785  FFF28D02B1EACBE1              1

[2786 rows x 2 columns]
Número de usuarios que aparecen tanto en grupo A como B: 0
Empty DataFrame
Columns: [user_id, groups_number]
Index: []


Observaciones: 
1. Ningún usuario fue asignado a ambos grupos de prueba A y B, por lo tanto, no existe contaminación de usuarios entre el grupo A y el grupo B.
2. El número previsto de participantes en la prueba es 6,000. Sin embargo, al sumar los usuarios que participaron en el grupo A (2120) con los usuarios que participaron en el grupo B (666), se tiene una muestra de tan solo 2,786 participantes.
3. El número de usuarios en el grupo A es más de 3 veces el número de usuarios en el grupo B. No obstante, se utilizará la prueba Z para proporciones la cual está diseñada para manejar bases de usuarios de diferentes tamaños. 

In [109]:
# Segmentar el DataFrame en grupo de prueba A y grupo de prueba B para el análisis del embudo de conversión:
# a. Dividir al grupo A:
df_group_A = df_events_14_days[df_events_14_days["group"] == "A"]
# b. Dividir al grupo B:
df_group_B = df_events_14_days[df_events_14_days["group"] == "B"]

# Mostras los primeros 10 registros de cada grupo de prueba. 
print("Muestra del grupo de prueba A: ")
print(df_group_A.head(10))
print()
print("Muestra del grupo de prueba B: ")
print(df_group_B.head(10))

Muestra del grupo de prueba A: 
             user_id first_date region   device            event_dt  \
0   D72A72121175D8BE 2020-12-07     EU       PC 2020-12-07 21:52:10   
7   831887FE7F2D6CBA 2020-12-07     EU  Android 2020-12-07 06:50:29   
8   831887FE7F2D6CBA 2020-12-07     EU  Android 2020-12-09 02:19:17   
9   831887FE7F2D6CBA 2020-12-07     EU  Android 2020-12-07 06:50:30   
10  831887FE7F2D6CBA 2020-12-07     EU  Android 2020-12-08 10:52:27   
11  831887FE7F2D6CBA 2020-12-07     EU  Android 2020-12-09 02:19:17   
15  3C5DD0288AC4FE23 2020-12-07     EU       PC 2020-12-07 19:42:40   
16  3C5DD0288AC4FE23 2020-12-07     EU       PC 2020-12-07 19:42:40   
17  B27F5380AF0936D3 2020-12-07     EU  Android 2020-12-07 07:40:27   
18  B27F5380AF0936D3 2020-12-07     EU  Android 2020-12-08 17:32:18   

      event_name  details group                  ab_test  \
0   product_page      NaN     A  recommender_system_test   
7       purchase     4.99     A  recommender_system_test   
8     

In [110]:
# Obtener el total de usuarios únicos por grupo.
total_unique_users_A = df_group_A["user_id"].nunique()
total_unique_users_B = df_group_B["user_id"].nunique()

print(f"Total de usuarios únicos - Grupo A: {total_unique_users_A} usuarios.")
print(f"Total de usuarios únicos - Grupo B: {total_unique_users_B} usuarios.")

Total de usuarios únicos - Grupo A: 2120 usuarios.
Total de usuarios únicos - Grupo B: 666 usuarios.


In [111]:

# Contar el número de usuarios únicos del grupo A en cada etapa del embudo:
# a. product_page.
product_page_users_A = df_group_A[df_group_A["event_name"] == "product_page"]["user_id"].nunique()
# b. product_cart.
product_cart_users_A = df_group_A[df_group_A["event_name"] == "product_cart"]["user_id"].nunique()
# c. purchase.
purchase_users_A = df_group_A[df_group_A["event_name"] == "purchase"]["user_id"].nunique()

print("Embudo de conversión - Grupo A: ")
print(f"product_page: {product_page_users_A} usuarios.")
print(f"product_cart: {product_cart_users_A} usuarios.")
print(f"purchase: {purchase_users_A} usuarios.")


Embudo de conversión - Grupo A: 
product_page: 1685 usuarios.
product_cart: 782 usuarios.
purchase: 833 usuarios.


In [112]:

# Contar el número de usuarios únicos del grupo B en cada etapa del embudo:
# a. product_page.
product_page_users_B = df_group_B[df_group_B["event_name"] == "product_page"]["user_id"].nunique()
# b. product_cart.
product_cart_users_B = df_group_B[df_group_B["event_name"] == "product_cart"]["user_id"].nunique()
# c. purchase.
purchase_users_B = df_group_B[df_group_B["event_name"] == "purchase"]["user_id"].nunique()

print("Embudo de conversión - Grupo B: ")
print(f"product_page: {product_page_users_B} usuarios.")
print(f"product_cart: {product_cart_users_B} usuarios.")
print(f"purchase: {purchase_users_B} usuarios.")


Embudo de conversión - Grupo B: 
product_page: 493 usuarios.
product_cart: 244 usuarios.
purchase: 249 usuarios.


Observaciones: 
1. Para el grupo de prueba A, 51 usuarios compraron sin un evento product_cart registrado.
2. Para el grupo de prueba B, 5 usuarios compraron sin un evento product_cart registrado.
3. Para ambos grupos de prueba A y B se tienen más usuarios en la etapa de purchase que en product_cart, es decir, existen usuarios que pasan directamente del producto a la compra saltándose el evento del carrito. 
4. La secuencia del embudo de conversión asumido product_page_product_cart -> purchase parece no ser obligatorio. Aparentemente existen casos en los que el embudo se reduce a product_page -> purchase. 

In [113]:
# Calcular las tasas de conversión para el grupo A.
# a. Tasa de conversión de product_page a product_cart.
conversion_page_to_cart_A = product_cart_users_A / product_page_users_A
# b. Tasa de conversión de product_cart a purchase. 
conversion_cart_to_purchase_A = purchase_users_A / product_cart_users_A

print("Tasas de conversión - Grupo A:")
print(f"Product Page → Product Cart: {conversion_page_to_cart_A:.4f} ({conversion_page_to_cart_A*100:.2f}%)")
print(f"Product Cart → Purchase: {conversion_cart_to_purchase_A:.4f} ({conversion_cart_to_purchase_A*100:.2f}%)")


Tasas de conversión - Grupo A:
Product Page → Product Cart: 0.4641 (46.41%)
Product Cart → Purchase: 1.0652 (106.52%)


In [114]:
# Calcular las tasas de conversión para el grupo B.
# a. Tasa de conversión de product_page a product_cart.
conversion_page_to_cart_B = product_cart_users_B / product_page_users_B
# b. Tasa de conversión de product_cart a purchase.
conversion_cart_to_purchase_B = purchase_users_B / product_cart_users_B

print("Tasas de conversión - Grupo B:")
print(f"Product Page → Product Cart: {conversion_page_to_cart_B:.4f} ({conversion_page_to_cart_B*100:.2f}%)")
print(f"Product Cart → Purchase: {conversion_cart_to_purchase_B:.4f} ({conversion_cart_to_purchase_B*100:.2f}%)")

Tasas de conversión - Grupo B:
Product Page → Product Cart: 0.4949 (49.49%)
Product Cart → Purchase: 1.0205 (102.05%)


In [115]:
# Crear un resumen comparativo:
# a. Diferencia absoluta entre grupos A y B.
difference_page_to_cart = conversion_page_to_cart_B - conversion_page_to_cart_A
difference_cart_to_purchase = conversion_cart_to_purchase_B - conversion_cart_to_purchase_A

print("Resumen comparativo:")
print(f"Product Page → Product Cart: {difference_page_to_cart:.4f}. Mejora: {difference_page_to_cart*100:.2f}%")
print(f"Product Cart → Purchase: {difference_cart_to_purchase:.4f}. Mejora: {difference_cart_to_purchase*100:.2f}%")

Resumen comparativo:
Product Page → Product Cart: 0.0308. Mejora: 3.08%
Product Cart → Purchase: -0.0447. Mejora: -4.47%


Observaciones: 

Al comparar las tasas del embudo de conversión de ambos grupos de prueba A y B, se tiene que:
1. El grupo B tiene un aumento del 3.08% en la conversión de la página del producto al carrito de compras (product_page -> product_cart) en comparación con el grupo A. 
2. El grupo B tiene un decremento del 4.47% en la conversión del carrito a la compra (product_cart -> purchase) en comparación con el grupo. 
3. En ninguna etapa del embudo de conversión se consigue un aumento de al menos un 10% como se pretendía.

Para un análisis completo y válido:
1. El embudo de conversión debe simplificarse a product_page -> purchase, es decir, comparar la proporción de los usuarios únicos que vieron la página del producto con la proporción de usuarios que compraron.
2. La prueba de proporciones Z se debe centrar en comparar la tasa de conversión total. La tasa de conversión total se define como:
usuarios que compran / usuarios que ven la página del producto. 

In [116]:

# Calcular la tasa de conversión total de product_page a purchase:
# Grupo A.
conversion_page_to_purchase_A = purchase_users_A / product_page_users_A
# Grupo B.
conversion_page_to_purchase_B = purchase_users_B / product_page_users_B

print("Tasa de conversión total - Grupo A:")
print(f"Product Page → Purchase: {conversion_page_to_purchase_A:.4f} ({conversion_page_to_purchase_A*100:.2f}%)")
print()
print("Tasa de conversión total - Grupo B:")
print(f"Product Page → Purchase: {conversion_page_to_purchase_B:.4f} ({conversion_page_to_purchase_B*100:.2f}%)")


Tasa de conversión total - Grupo A:
Product Page → Purchase: 0.4944 (49.44%)

Tasa de conversión total - Grupo B:
Product Page → Purchase: 0.5051 (50.51%)


Observaciones: 

En la tasa de conversión total de la página del producto a la compra (product_page -> purchase):
1. El grupo de prueba B tiene un aumento de conversión del 1.07% en comparación con el grupo A.
2. El aumento de conversión es menor al 10% esperado. 

## 5. Prueba de proporciones Z

In [117]:
# Definir los parámetros de:
# a. Usuarios únicos del grupo A que completaron la acción de compra (purchase). Estos son los éxitos de conversión del grupo A.
# b. Usuarios únicos del grupo A que vieron la página del producto (product_page). Estos representan el número de observaciones totales del grupo A. 
# c. Usuarios únicos del grupo B que completaron la acción de compra (purchase). Estos son los éxitos de conversión del grupo B.
# d. Usuarios únicos del grupo B que vieron la página del producto (product_page). Estos representan el número de observaciones totales del grupo B. 
purchase_users_A = 833
product_users_A = 1685
purchase_users_B = 249
product_users_B = 493

# Crear un arreglo con los usuarios que completaron la acción de compra en grupos B y A. 
# Colocar primero al grupo B ya que se desea comprobar que B > A
purchase_users = np.array([purchase_users_B, purchase_users_A])
product_users = np.array([product_users_B, product_users_A])

# Establecer el nivel de significancia estadística crítica
# Si el valor-p es menor que alpha se rechaza la hipótesis
alpha = 0.05

# Ejecutar prueba Z. 
# Utilizar el parámetro alternative = "larger" para indicar que la hipótesis alternativa implica que la proporción del grupo B es mayor que la del grupo A. 
z_stat, pvalue = proportions_ztest(purchase_users, product_users, alternative = "larger")
print(f'Estadístico Z: {z_stat:.4f}')
print(f'Valor p: {pvalue:.4f}')

# Probar la hipótesis:
# Hipótesis nula: El sistema de recomendaciones mejorado no es mejor que el original.
# Hipótesis alternativa: El sistema de recomendaciones mejorado es significativamente mejor que el original.
if (pvalue < alpha):
    print("Rechazamos la hipótesis nula.")
    print("El sistema de recomendaciones mejorado es significativamente mejor.")
else:
    print("No podemos rechazar la hipótesis nula.")
    print("No hay evidencia suficiente de que el sistema de recomendaciones mejorado sea mejor.")

Estadístico Z: 0.4183
Valor p: 0.3379
No podemos rechazar la hipótesis nula.
No hay evidencia suficiente de que el sistema de recomendaciones mejorado sea mejor.


## 6. Conclusiones

La prueba A/B "recommender_system_test" fue comprometida por errores de diseño:
1. El número total de participantes fue de 2,786 usuarios en la región EU registrados entre el 07 y el 21 de diciembre de 2020, lo que está lejos de los 6,000 esperados.
2. La distribución del tráfico fue desigual. El grupo de control A era tres veces más grande que el grupo de prueba B (2120 usuarios vs 666).
3. El conteo de usuarios únicos mostró que la cantidad de usuarios que hicieron una compra es mayor que la cantidad de usuarios que agregaron al carrito en ambos grupos. En consecuencia, el embudo secuencial product_page -> product_cart -> purchase es inválido. Debido a esto, el análisis se centró en la conversión válida product_page -> purchase para tomar la decisión final. 

Con base en el rendimiento observado en el embudo de eventos product_page -> purchase, el grupo de prueba B mostró una mejora marginal del 1.07% respecto al grupo de control A. Por ende, el objetivo de negocio de la tienda en línea internacional no se ha cumplido. La mejora del 1.07% está significativamente por debajo del objetivo del 10% de aumento esperado. 

No existe evidencia suficiente y estadísticamente significativa para afirmar que el nuevo sistema de recomendaciones es mejor que el sistema de control. Los problemas de diseño como la baja muestra, el desequilibrio en los grupos de prueba, etc. provocan que cualquier resultado positivo potencial sea imposible de validar con los datos actuales. 

La recomendación final sería que el nuevo sistema de recomendaciones NO SE IMPLEMENTE. 